## Neuro-boosted models and probabilistic forecast

The strength of inait forecasting models lies in the connection with digital brains. In this example we will show how to use more advanced models. In the second part we will present probabilistic forecasting which provides a distribution of possible outcomes associated with probabilities, allowing the user and stakeholders to take better decisions under uncertainty. 

### Prerequisites

In [ ]:
# Add parent directory to Python path for client imports
import sys
import os
sys.path.append(os.path.join(os.path.dirname(os.getcwd())))

import pandas as pd 

from client.prediction_script import (
    create_payload_from_file,
    get_dataframe_from_response,
)
from client.utils import make_request
# 🔧 API Configuration
base_url = "https://"       # Add the URL of your inait forecast deployment/SaaS
auth_key = "your_key_here"  # Replace with your actual API key

# 🛡️ Validate authentication setup
if not auth_key:
    raise ValueError("❌ Authentication key is required. Please check your API credentials.")

if not base_url:
    raise ValueError("❌ Base URL is required. Please check your API endpoint.")

### Split the data into train/test 

For this example we use another classic dataset: the Etth1. This has hourly granularity and 7 variables from loads to temperatures etc. We will predict all 7 quantities for the next 4 days, i.e., a forecasting horizon of 24 steps. 

In [ ]:
### Load and visualize the data 

data_path = "../data/autoformer_etth1.csv"
data = pd.read_csv(data_path, index_col=0)
data.columns = [col.strip('__target') for col in data.columns]  # Clean column names
display(data.head())

# Configure prediction parameters
forecasting_horizon = 24  # Predict 24 time steps ahead (note: 96 steps is the default for the ETTh1 dataset but to speed up the example, we will use 24 steps)
observation_length = 94   # Use last 24 points as historical context

# Create the prediction payload
# This packages your data and parameters into the format expected by the API
payload = create_payload_from_file(
    file_path=data_path,
    target_columns= ",".join(data.columns.tolist()), # Use all columns as targets
    forecasting_horizon=forecasting_horizon,
    observation_length=observation_length,
)

# Send prediction request to the API
response = make_request( base_url + "/prediction", payload, auth_key=auth_key)

# Process the response and extract results
df, session_id = get_dataframe_from_response(response)

# Display the prediction results
display(df)